# Special Methods: Delete 

The `__del__` method will get called right before the object is __destroyed__ by the Garbage Collector

__We do not control when it will get called!__

Exceptions in `__dell__` are __silent__

Prefer using context managers to clean up conections and resources

In [6]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [1]:
class Person:
    def __init__(self, name):
        self.name = name 

    def __repr__(self) -> str:
        return f"Person({self.name})"

    def __del__(self):
        print(f'__del__ called for {self}')

In [4]:
p = Person('Lorena') # First reference to object 

__del__ called for Person(Lorena)


In [7]:
id_p = id(p)
ref_count(id_p)

1

In [8]:
p = None

__del__ called for Person(Lorena)


In [ ]:
p = Person('Gabriel')
del p

__del__ called for Person(Gabriel)


In [11]:
class Person:
    def __init__(self, name):
        self.name = name 

    def __repr__(self) -> str:
        return f"Person({self.name})"

    def __del__(self):
        print(f'__del__ called for {self}')

    def generate_exception(self):
        raise ValueError('Exception atificially generated')

In [12]:
p = Person('Nilo')
p_id = id(p)

ref_count(p_id)

1

In [15]:
try:
    p.generate_exception()
except ValueError as ex:
    error = ex 
    print(ex)

Exception atificially generated


In [16]:
ex

NameError: name 'ex' is not defined

In [17]:
error

ValueError('Exception atificially generated')

In [18]:
ref_count(p_id)

2

In [19]:
dir(error)

['__cause__',
 '__class__',
 '__context__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__suppress_context__',
 '__traceback__',
 'args',
 'with_traceback']

In [20]:
type(error.__traceback__.tb_frame.f_locals)

dict

In [21]:
for key, value in error.__traceback__.tb_frame.f_locals.items():
    print(key, value)

__name__ __main__


RuntimeError: dictionary changed size during iteration

In [25]:
for key, value in error.__traceback__.tb_frame.f_locals.copy().items():
    if isinstance(value, Person):
        print(key, value, id(value))

p Person(Nilo) 1945747688560
value Person(Nilo) 1945747688560


In [26]:
ref_count(p_id)

2

In [27]:
del p

In [28]:
del error

In [29]:
del p

NameError: name 'p' is not defined

In [30]:
ref_count(p_id)

1

We don't know where or why the referece still there D: 
That's why its not usual to implement `__del__` method, because in fact we do not have control

In [31]:
class Person:
    def __del__(self):
        raise ValueError('Something went bump...')

In [33]:
p = Person()
del p

Exception ignored in: <function Person.__del__ at 0x000001C507966A70>
Traceback (most recent call last):
  File "C:\Users\loris\AppData\Local\Temp\ipykernel_21676\795576010.py", line 3, in __del__
ValueError: Something went bump...
Exception ignored in: <function Person.__del__ at 0x000001C507966A70>
Traceback (most recent call last):
  File "C:\Users\loris\AppData\Local\Temp\ipykernel_21676\795576010.py", line 3, in __del__
ValueError: Something went bump...


In [34]:
import sys 

In [35]:
sys.stderr, sys.stdout

(<ipykernel.iostream.OutStream at 0x1c506d00040>,
 <ipykernel.iostream.OutStream at 0x1c506d016f0>)

In [47]:
class ErrToFile:
    def __init__(self, fname):
        self._fname = fname 
        self._current_stderr = sys.stderr
    
    def __enter__(self):
        self._file = open(self._fname, 'w')
        sys.stderr = self._file

    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stderr = self._current_stderr
        if self._file:
            self._file.close()
        return False 

In [43]:
p = Person()

Exception ignored in: <function Person.__del__ at 0x000001C507966A70>
Traceback (most recent call last):
  File "C:\Users\loris\AppData\Local\Temp\ipykernel_21676\795576010.py", line 3, in __del__
ValueError: Something went bump...


In [ ]:
with ErrToFile('err.txt'):
    del p 
    print(100)

print(200)
print(300)

100
200
300
